In [ ]:
import random
import queue

# Linear Search

- $O(n)$

In [ ]:
def linear_search(data, val):
    process_count = 0
    for i, datum in enumerate(data):
        process_count += 1
        if datum == val:
            return True, i, process_count
    return False, None, process_count

data = list(range(1000))
random.shuffle(data)

print(linear_search(data, 0))
print(linear_search(data, 500))
print(linear_search(data, 999))
print(linear_search(data, 1000))

# Binary Search

- $O(\log_2{n})$

ソートしておけば中央値を見て片側に(半分に)絞っていけるじゃん、というシンプルで強力な考え

## recursive

- 意外と再帰使う方が複雑なコードになってしまった、、これなら再帰使わない方がスマートかも、、

In [ ]:
def binary_search(data, val, process_count = 0):
    """data must be sorted"""
    process_count += 1
    center_index = len(data) // 2
    
    if len(data) == 0:
        return False, None, process_count
    if center_index == 0:
        if val == data[center_index]:
            return True, center_index, process_count
        return False, None, process_count
    
    if val < data[center_index]:
        return binary_search(data[:center_index], val, process_count)
    elif data[center_index] < val:
        is_found, index, count = binary_search(data[center_index + 1:], val, process_count)
        return is_found, center_index + 1 + index if is_found else None, count
    else:
        return True, center_index, process_count

data = list(range(1000))

for _ in range(3):
    val_to_search = random.randint(0, 999)
    print(val_to_search, binary_search(data, val_to_search))

print(binary_search(data, 1000))

## not recursive

In [ ]:
def binary_search(data, val):
    """data must be sorted"""
    process_count = 0
    start = 0
    end = len(data) - 1
    
    while start <= end:
        process_count += 1
        center = (start + end) // 2
        if val < data[center]:
            end = center - 1
        elif data[center] < val:
            start = center + 1
        else:
            return True, center, process_count
    return False, None, process_count

data = list(range(1000))

for _ in range(3):
    val_to_search = random.randint(0, 999)
    print(val_to_search, binary_search(data, val_to_search))

print(binary_search(data, 1000))

# Tree Search

In [ ]:
class Node(object):
    def __init__(self, val):
        self.val = val
        self.children = []
        self.level = 0

    def add_child(self, node):
        node.level = self.level + 1
        self.children.append(node)

values = list(range(100))
random.shuffle(values)
value_queue = queue.Queue()
for v in values:
    value_queue.put(v)

def add_children_to_one_node(parent_node, min_children=1, max_children=4):
    num_of_children = random.randint(min_children, max_children)
    for _ in range(num_of_children):
        if value_queue.empty():
            return
        parent_node.add_child(Node(value_queue.get_nowait()))

# add chilren by breadth first way, to create a balanced_tree-like tree
def add_children_balancely(parent_node):
    q = queue.Queue()
    q.put(parent_node)
    while not q.empty():
        node = q.get_nowait()
        add_children_to_one_node(node)
        if value_queue.empty():
            return
        for child in node.children:
            q.put(child)

# this uses recursive way to add children, but this results in creating unbalanced treek
def add_children_deeply(parent_node):
    add_children_to_one_node(parent_node)
    for child in parent_node.children:
        add_children(child)
        if value_queue.empty():
            return

root_node = Node(value_queue.get_nowait())
add_children_balancely(root_node)
# add_children_deeply(root_node)

def print_node(node):
    print("-" * node.level + str(node.val))
    for child in node.children:
        print_node(child)

print_node(root_node)

## Depth First Search

- use recursive or stack
- advantage
    - good memory performance
    - 一筆書きできる
- disadvanage
    - takes time to exploring deep areas

### use recursive way

In [ ]:
def depth_first_search(root_node, val, print_=True):
    trace_stack = queue.LifoQueue()
    def search(node):
        if print_:
            print("-" * node.level + str(node.val))
        trace_stack.put(node)
        if node.val == val:
            return True
        for child in node.children:
            found = search(child)
            if found:
                return True
        trace_stack.get_nowait()
        return False

    found = search(root_node)

    if not found:
        return False, None
    traces = []
    while not trace_stack.empty():
        node = trace_stack.get_nowait()
        traces.append(node)
    traces.reverse()
    return True, traces


found, traces = depth_first_search(root_node, 50)
print("========")
print(found)
if found:
    for node in traces:
        print(node.val)

### use stack way

In [ ]:
# 再帰の時のようにtraceをうまいこと残せない、、、もっと考えればうまい方法があるかも
def depth_first_search(root_node, val, print_=True):
    stack = queue.LifoQueue()
    stack.put(root_node)
    while not stack.empty():
        node = stack.get_nowait()
        if print_:
            print("-" * node.level + str(node.val))
        if node.val == val:
            return True
        for child in reversed(node.children):
            stack.put(child)
    return False

found = depth_first_search(root_node, 50)
print("========")
print(found)

## Breadth First Search

- use queue
- advantage
    - okey if there are deep areas
- disadvantage
    - not good memory performance

In [ ]:
def breadth_first_search(root_node, val, print_=True):
    q = queue.Queue()
    q.put(root_node)
    while not q.empty():
        node = q.get_nowait()
        if print_:
            print("-" * node.level + str(node.val))
        if node.val == val:
            return True
        for child in node.children:
            q.put(child)
    return False

found = breadth_first_search(root_node, 50)
print("========")
print(found)

# Sort

### stable sort

- 安定ソート
- maintain the relative order of records with equal keys

### in-place

- 内部ソート
- good memory perfomance: do not use other computer resources
    - sort algorithm needing only $O(1)$ or $O(\log{n})$ of other resources is considered "in-place"
    - directly edit the memory region of data to be sorted, so this kind of algorithm does not need other resources.


# Insertion Sort

slow but in-place and stable sort.  
very quick for almost sorted data.

- $O(n^{2})$
- stable sort
- in-place
- $O(n)$ for almost sorted data

In [ ]:
def insertion_sort(data):
    process_count = 0
    for target_index in range(1, len(data)):
        target_value = data[target_index]
        j = target_index - 1
        while 0 <= j and target_value < data[j]:
            process_count += 1
            data[j + 1] = data[j]
            j -= 1
        data[j + 1] = target_value
    return process_count

for _ in range(5):
    data = list(range(1000))
    random.shuffle(data)
    process_count = insertion_sort(data)
    print(process_count)

print("==for almost sorted data==")
for _ in range(5):
    data = list(range(1000))
    random.shuffle(data)
    partial = data[:995]
    insertion_sort(partial)
    data = partial + data[995:]
    process_count = insertion_sort(data)
    print(process_count)


# Merge Sort

Quick stable sort and very popular as well as Quick-Sort.  
This is also called "しめじソート"

- $O(n\log{n})$
    - same in the worst case
- stable sort
- **NOT** in-place

In [ ]:
def merge_sort(entire_data):
    process_count = 0
    def sort(data):
        nonlocal process_count
        process_count += 1
        if len(data) <= 1:
            return
        center_index = len(data) // 2
        left = data[:center_index]
        right = data[center_index:]

        sort(left)
        sort(right)

        l_i = 0
        r_i = 0

        def use_right():
            nonlocal r_i
            data[i] = right[r_i]
            r_i += 1
        
        def use_left():
            nonlocal l_i
            data[i] = left[l_i]
            l_i += 1
        
        for i in range(len(data)):
            process_count += 1
            if len(left) <= l_i:
                use_right()
                continue
            if len(right) <= r_i:
                use_left()
                continue
            
            if left[l_i] < right[r_i]:
                use_left()
            else:
                use_right()
    sort(entire_data)
    return process_count

for _ in range(5):
    data = list(range(1000))
    random.shuffle(data)
    process_count = merge_sort(data)
    print(process_count)

print("==for almost sorted data==")
for _ in range(5):
    data = list(range(1000))
    random.shuffle(data)
    partial = data[:995]
    merge_sort(partial)
    data = partial + data[995:]
    process_count = merge_sort(data)
    print(process_count)

# Quick Sort

Quick in-place sort and very popular as well as Merge-Sort.

- $O(n\log{n})$
    - $O(n^2)$ in the worst case
- **NOT** stable sort
- in-place

挿入ソートが得意とするような、途中までほぼソートされているようなデータに対しては、  
pivot選択をランダムにしないと桁が変わるレベルで遅くなることがわかる。  
ランダムにすることで安定して $O(n\log{n})$ の速度を出せている。

In [ ]:
def quick_sort(data, randomize_pivot=False):
    process_count = 0
    def sort(start, end):
        nonlocal process_count
        if end <= start:
            return
        # The pivot should be selected randomly to get a stable speed.
        # In that case, swapping pivot and the end value is needed. (pivot must be in the end position.)
        if randomize_pivot:
            pivot_i = random.randint(start, end)
            data[pivot_i], data[end] = data[end], data[pivot_i]
        pivot = data[end]

        # i番目を確実にpivotよりも小さい数にしていく
        # それがjを[start,end)まで1回しするだけで可能というのが要所(前提: end番目がpivot値。ランダムに選択する場合は事前にendとswapしておく)
        # 仕組み
        # - 基本iとjは並進する
        # - i番目がすでにpivotよりも小さければ、i=jのまま同じ位置のswapが行われ、何も変化しないまま並進する
        # - i番目がpivotよりも大きい場合にjがiよりも進み、pivotよりも小さい値を見つけるまで進み続ける
        # - 見つかればi番目とj番目の値をswapすることで、やっとiも次に進める
        # - 次のiの処理のために、jを戻す必要はない。なぜならこの時点で[i,j]範囲にpivotよりも小さい値は存在しえないから。
        # - これをjがpivotに到達する前まで続ける。つまりjが[start,end)まで1回りする。
        # - 以上が完了すると、i番目はpivot以上の値(ずっと並進しつづければiもpivotまで到達し、その際はpivot値になる)なので、i番目とpivotをswapする
        # - 以上でi番目がpivotになり、[start,i-1]はすべてpivotより小さい値、[i+1,end]はすべてpivotより大きい値、に分割できたことになる
        i = start
        for j in range(start, end):
            process_count += 1
            if data[j] < pivot:
                data[i], data[j] = data[j], data[i]
                i += 1
        data[i], data[end] = data[end], data[i]
        sort(start, i - 1)
        sort(i + 1, end)
    sort(0, len(data) - 1)
    return process_count

for _ in range(5):
    data = list(range(1000))
    random.shuffle(data)
    process_count = quick_sort(data)
    print(process_count)

print("==for almost sorted data==")
for _ in range(5):
    data = list(range(1000))
    random.shuffle(data)
    partial = data[:995]
    quick_sort(partial)
    data = partial + data[995:]
    process_count = quick_sort(data)
    print(process_count)

print()
print("==randomized pivot==")
print()

for _ in range(5):
    data = list(range(1000))
    random.shuffle(data)
    process_count = quick_sort(data, randomize_pivot=True)
    print(process_count)

print("==for almost sorted data==")
for _ in range(5):
    data = list(range(1000))
    random.shuffle(data)
    partial = data[:995]
    quick_sort(partial)
    data = partial + data[995:]
    process_count = quick_sort(data, randomize_pivot=True)
    print(process_count)

# Heap Data Structure

- different concept from the "memory heap"
- a heap is a specialized tree-based data strucutre which satisfies the heap property.
    - the heap property
        - For any given node C, if P is a parent node of C, then the key(or the value) of P is greater than or equal to the key of C.
- A common implementation of a heap is the "binary heap", in which the tree is a binary tree.
- can be **represented in a array shape**
    - the children of a[i] are a[2i+1] and a[2i+2]
    - the parent of a[i] is a[(i-1)//2]
- can get the max value in $O(1)$, because a[0] is always the max value
- can remove the max value in $O(\log{n})$
    - swap a[0] with a[-1], and remove a[-1]
    - swap the larger of a[1] and a[2] with a[0], and do this recursively to the swapped child node
    - the height of the tree is $\log{n}$, so this process is finished in $O(\log{n})$

通常はin-placeにしたいので、配列で2分木を表現し、配列内のswapだけで処理を行うことになる。  
配列による2分木の表現方法を理解しておかないと何もわからないので注意。

## ヒープの構築方法

ヒープをfixする常套手法は、特定のノードについて再帰的に自分より大きい子ノードのうちより大きい方とswapを繰り返すこと。  
これは既に構築されたヒープから最大値、つまりルートノードを除去する際に、ルートノードに対して行われる処理であり、この状態であればこれだけでヒープの再構築が完了する。  
しかし一般的には、子ノードが自分よりも小さくなるように他のヒープを崩さず修正するだけに過ぎず、意図がつかみにくい処理に見えてしまう。  


以下のポイントを理解すれば、ロジックが腹に落ちるはず。

- 最大値を順番に取得するためにヒープを用いる文脈では、最大値を除去する処理が最重要
- ルートノード以外の調整や、ヒープを1から構築するには、この処理を後ろ側から繰り返すだけで良い
    - 後半は末端の子なしノードにきまっているので、 n//2 から 0 に向かって繰り返せば十分

後ろ側から繰り返せば全体をヒープにできる、というのは最初は納得しがたいかもしれないが、少なくとも1代しか子供を持たない末端親ノードについては、この処理を行うことでその位置以下の部分をヒープ構造にできたことになる(当たり前)。  
これを考えれば下から1段ずつ上に向かって処理を繰り返していけば、もれなくヒープ構造を広げていけるのは想像しやすいはず。  
頭から繰り返すとなぜダメなのか、ちゃんと考えればわかるかもだが、まあ確かに後ろからやった方がイメージしやすいというレベルの理解。。。

In [ ]:
# 他のヒープ構造を崩さず、特定のノードについて子ノードが自分よりも小さくなるよう構造を修正する、ヒープ最重要ロジック
# 末端親ノードからルートノードまで順番に行うことで全体をヒープにできる
# 既にヒープ構造の場合は、ルートノード、つまり最大値を除去した後の再構築に利用できる
# (末端のノードとルートをスワップし、スワップ済みのヒープが崩れたルートに対してこれを1回行えばいい)
# lengthを指定することで処理する領域を制限する。ソートに用いる場合に利用することになる。
def heap_one_node(data, i, length=None, process_count=0):
    if length is None:
        length = len(data)
    largest = i
    left = 2 * i + 1
    right = 2 * i + 2
    if left < length and data[i] < data[left]:
        largest = left
    if right < length and data[largest] < data[right]:
        largest = right
    if largest != i:
        data[i], data[largest] = data[largest], data[i]
        process_count = heap_one_node(data, largest, length, process_count + 1)
    return process_count

def heap_all(data):
    process_count = 0
    for i in range(len(data)//2, -1, -1):
        count = heap_one_node(data, i)
        process_count += count + 1
    return process_count

data = list(range(10))
random.shuffle(data)
heap_all(data)
print(data)

# Heap Sort

Quick in-place sort as well as Quick-Sort, but inferior to Quick-Sort in average speed.

- $O(n\log{n})$
    - same in the worst case
- **NOT** stable sort
- in-place

なぜかQuick-Sortよりも若干少ないループ回数がでた。本当はQuick-Sortの方が平均的には速いらしい。  
カウントの仕方か何かが間違っている可能性がある、、

In [ ]:
def heap_sort(data):
    process_count = 0
    count = heap_all(data)
    process_count += count
    for i in range(len(data) - 1, 0, -1):
        data[0], data[i] = data[i], data[0]
        count = heap_one_node(data, 0, i)
        process_count += count + 1
    return process_count


for _ in range(5):
    data = list(range(1000))
    random.shuffle(data)
    process_count = heap_sort(data)
    print(process_count)

print("==for almost sorted data==")
for _ in range(5):
    data = list(range(1000))
    random.shuffle(data)
    partial = data[:995]
    heap_sort(partial)
    data = partial + data[995:]
    process_count = heap_sort(data)
    print(process_count)

# Bucket Sort


# Selection Sort

# Bubble Sort